In [1]:
from keras import optimizers
from keras import losses
from keras import metrics
from keras import regularizers
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from keras.preprocessing import sequence

import numpy as np
from numpy import argmax

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# loading the data
import pandas as pd
data = pd.read_csv('data/SpeedDatingData.csv', engine='python')

In [3]:
labels = pd.DataFrame(data['match'])
data = data.drop(columns=['match'])

In [4]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
iid,8378.0,283.675937,158.583367,1.00,154.00,281.00,407.00,552.00
id,8377.0,8.960248,5.491329,1.00,4.00,8.00,13.00,22.00
gender,8378.0,0.500597,0.500029,0.00,0.00,1.00,1.00,1.00
idg,8378.0,17.327166,10.940735,1.00,8.00,16.00,26.00,44.00
condtn,8378.0,1.828837,0.376673,1.00,2.00,2.00,2.00,2.00
wave,8378.0,11.350919,5.995903,1.00,7.00,11.00,15.00,21.00
round,8378.0,16.872046,4.358458,5.00,14.00,18.00,20.00,22.00
position,8378.0,9.042731,5.514939,1.00,4.00,8.00,13.00,22.00
positin1,6532.0,9.295775,5.650199,1.00,4.00,9.00,14.00,22.00
order,8378.0,8.927668,5.477009,1.00,4.00,8.00,13.00,22.00


In [5]:
# split the data into train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.30)

In [7]:
print(len(x_train))
print(len(y_train))

5864
5864
